In [ ]:
!{sys.executable} -m pip install boto3

In [ ]:
import boto3

iam_client = boto3.client('iam',
  endpoint_url='http://s3.default.svc.cluster.local',
  region_name=''
)

## Admin

* AssumeRole admin, by named principals (example: Tom)
* Owns all buckets
* Creates and modifies IAM policies, IAM OIDC providers

In [ ]:
policy_document = '''{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Principal":{"AWS":["arn:aws:iam:::user/zippy"]},"Action":["sts:AssumeRole"]}]}'''

role_response = iam_client.create_role(
  AssumeRolePolicyDocument=policy_document,
  Path='/',
  RoleName='Admin',
)

# Need to add allowances for IAM resources
role_policy = '''{"Version":"2012-10-17","Statement":{"Effect":"Allow","Action":"s3:*","Resource":"arn:aws:s3:::*"}}'''

response = iam_client.put_role_policy(
  RoleName='Admin',
  PolicyName='AdminPolicy',
  PolicyDocument=role_policy
)

Create red, blue, and green buckets

## Data Engineers

* AssumeRole lakeadmin, by named principals (example: Dick)
* Creates and modifies bucket policy
* Creates and modifies bucket lifecycle configurations
* Cleans data with ETL jobs, etc.

In [ ]:
policy_document = '''{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Principal":{"AWS":["arn:aws:iam:::user/zippy"]},"Action":["sts:AssumeRole"]}]}'''

role_response = iam_client.create_role(
  AssumeRolePolicyDocument=policy_document,
  Path='/',
  RoleName='DataEngineer',
)

# Need to reduce this
## PutObject, PutBucketPolicy, PutBucketLifecycleConfiguration, GetObject, GetObjectContents
role_policy = '''{"Version":"2012-10-17","Statement":{"Effect":"Allow","Action":"s3:*","Resource":"arn:aws:s3:::*"}}'''

response = iam_client.put_role_policy(
  RoleName='DataEngineer',
  PolicyName='DataEngineerPolicy',
  PolicyDocument=role_policy
)

## Interactive Query (Trino)

* AssumeRole presto, by service user
* Policy enforment point for named principals without direct access (example: Harry)
* Integrated with Apache Ranger

In [ ]:
policy_document = '''{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Principal":{"AWS":["arn:aws:iam:::user/trino"]},"Action":["sts:AssumeRole"]}]}'''

role_response = iam_client.create_role(
  AssumeRolePolicyDocument=policy_document,
  Path='/',
  RoleName='Trino',
)

# Need to reduce this
## permit GetObject, GetObjectContents
role_policy = '''{"Version":"2012-10-17","Statement":{"Effect":"Allow","Action":"s3:*","Resource":"arn:aws:s3:::*"}}'''

response = iam_client.put_role_policy(
  RoleName='Trino',
  PolicyName='TrinoPolicy',
  PolicyDocument=role_policy
)


Example Trino configuration

```
hive.s3.iam-role=Trino
hive.s3.sts.endpoint=http://s3.default.svc.cluster.local
hive.s3.sts.region=''
hive.s3.aws-access-key=
hive.s3.aws-secret-key=
hive.s3.endpoint=http://s3.default.svc.cluster.local
hive.s3.path-style-access=true
hive.s3.ssl.enabled=false
hive.s3select-pushdown.enabled=true
```

# Workflows

Assume role LakeAdmin

In [ ]:
sts_client = boto3.client('sts',
  aws_access_key_id=??,
  aws_secret_access_key=??,
  endpoint_url='http://s3.default.svc.cluster.local',
  region_name='',
)

response = sts_client.assume_role(
  RoleArn=role_response['Role']['Arn'],
  RoleSessionName='Bob',
  DurationSeconds=3600
)

In [ ]:
s3client = boto3.client('s3',
  aws_access_key_id = response['Credentials']['AccessKeyId'],
  aws_secret_access_key = response['Credentials']['SecretAccessKey'],
  aws_session_token = response['Credentials']['SessionToken'],
  endpoint_url='http://s3.default.svc.cluster.local',
  region_name='',)

bucket_name = 'my-bucket'
s3bucket = s3client.create_bucket(Bucket=bucket_name)
resp = s3client.list_buckets()